In [ ]:
import pandas as pd
import numpy as np

In [2]:
import pandas as pd # Importing the pandas library
import numpy as np

# prompt: upload rthe csv

from google.colab import files
uploaded = files.upload()
# Assuming the uploaded file is named 'your_file.csv'
# Replace 'your_file.csv' with the actual file name if it's different
df = pd.read_csv('Airbnb_Open_Data.csv') # Now pd is recognized
df


Saving Airbnb_Open_Data.csv to Airbnb_Open_Data (1).csv


<ipython-input-2-2426618f484f>:10: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Airbnb_Open_Data.csv') # Now pd is recognized


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102594,6092437,Spare room in Williamsburg,12312296767,verified,Krik,Brooklyn,Williamsburg,40.70862,-73.94651,United States,...,$169,1.0,0.0,NaN,NaN,3.0,1.0,227.0,No Smoking No Parties or Events of any kind Pl...,NaN
102595,6092990,Best Location near Columbia U,77864383453,unconfirmed,Mifan,Manhattan,Morningside Heights,40.80460,-73.96545,United States,...,$167,1.0,1.0,7/6/2015,0.02,2.0,2.0,395.0,House rules: Guests agree to the following ter...,NaN
102596,6093542,"Comfy, bright room in Brooklyn",69050334417,unconfirmed,Megan,Brooklyn,Park Slope,40.67505,-73.98045,United States,...,$198,3.0,0.0,NaN,NaN,5.0,1.0,342.0,NaN,NaN
102597,6094094,Big Studio-One Stop from Midtown,11160591270,unconfirmed,Christopher,Queens,Long Island City,40.74989,-73.93777,United States,...,$109,2.0,5.0,10/11/2015,0.10,3.0,1.0,386.0,NaN,NaN


In [ ]:
# df = pd.read_csv('/kaggle/input/airbnbopendata/Airbnb_Open_Data.csv')

In [3]:
df.head()

,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


# *Renaming the columns*

In [4]:
df.columns = df.columns.str.upper()

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102599 entries, 0 to 102598
Data columns (total 26 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ID                              102599 non-null  int64  
 1   NAME                            102349 non-null  object 
 2   HOST ID                         102599 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          102310 non-null  object 
 4   HOST NAME                       102193 non-null  object 
 5   NEIGHBOURHOOD GROUP             102570 non-null  object 
 6   NEIGHBOURHOOD                   102583 non-null  object 
 7   LAT                             102591 non-null  float64
 8   LONG                            102591 non-null  float64
 9   COUNTRY                         102067 non-null  object 
 10  COUNTRY CODE                    102468 non-null  object 
 11  INSTANT_BOOKABLE                102494 non-null  object 
 12  CANCELLATION_POL

In [6]:
df.columns

Index(['ID', 'NAME', 'HOST ID', 'HOST_IDENTITY_VERIFIED', 'HOST NAME',
       'NEIGHBOURHOOD GROUP', 'NEIGHBOURHOOD', 'LAT', 'LONG', 'COUNTRY',
       'COUNTRY CODE', 'INSTANT_BOOKABLE', 'CANCELLATION_POLICY', 'ROOM TYPE',
       'CONSTRUCTION YEAR', 'PRICE', 'SERVICE FEE', 'MINIMUM NIGHTS',
       'NUMBER OF REVIEWS', 'LAST REVIEW', 'REVIEWS PER MONTH',
       'REVIEW RATE NUMBER', 'CALCULATED HOST LISTINGS COUNT',
       'AVAILABILITY 365', 'HOUSE_RULES', 'LICENSE'],
      dtype='object')

# *Deleting the reduntant columns*

In [7]:
columns_to_keep = ['ID', 'NAME', 'HOST ID', 'HOST_IDENTITY_VERIFIED', 'HOST NAME',
       'NEIGHBOURHOOD GROUP', 'NEIGHBOURHOOD', 'LAT', 'LONG', 'COUNTRY',
       'COUNTRY CODE', 'INSTANT_BOOKABLE', 'CANCELLATION_POLICY', 'ROOM TYPE',
       'CONSTRUCTION YEAR', 'PRICE', 'SERVICE FEE', 'MINIMUM NIGHTS',
       'NUMBER OF REVIEWS','REVIEW RATE NUMBER', 'CALCULATED HOST LISTINGS COUNT','AVAILABILITY 365']

columns_to_drop = ['LICENSE','HOUSE_RULES','LAST REVIEW','REVIEWS PER MONTH']

In [8]:
df.drop(columns=columns_to_drop,inplace=True)

In [9]:
df.columns == columns_to_keep

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [10]:
df.head()

,ID,NAME,HOST ID,HOST_IDENTITY_VERIFIED,HOST NAME,NEIGHBOURHOOD GROUP,NEIGHBOURHOOD,LAT,LONG,COUNTRY,...,CANCELLATION_POLICY,ROOM TYPE,CONSTRUCTION YEAR,PRICE,SERVICE FEE,MINIMUM NIGHTS,NUMBER OF REVIEWS,REVIEW RATE NUMBER,CALCULATED HOST LISTINGS COUNT,AVAILABILITY 365
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,strict,Private room,2020.0,$966,$193,10.0,9.0,4.0,6.0,286.0
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,moderate,Entire home/apt,2007.0,$142,$28,30.0,45.0,4.0,2.0,228.0
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,flexible,Private room,2005.0,$620,$124,3.0,0.0,5.0,1.0,352.0
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,moderate,Entire home/apt,2005.0,$368,$74,30.0,270.0,4.0,1.0,322.0
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0,3.0,1.0,289.0


# *Dropping the duplicates*

In [11]:
df.duplicated().sum()

np.int64(541)

In [12]:
df.drop_duplicates(inplace=True)

In [13]:
df.duplicated().sum()

np.int64(0)

# *Removing the NaN values*

In [14]:
df.isna().sum()

,0
ID,0
NAME,250
HOST ID,0
HOST_IDENTITY_VERIFIED,289
HOST NAME,404
NEIGHBOURHOOD GROUP,29
NEIGHBOURHOOD,16
LAT,8
LONG,8
COUNTRY,532


In [15]:
df.dropna(inplace=True)

In [16]:
df.isna().sum()

,0
ID,0
NAME,0
HOST ID,0
HOST_IDENTITY_VERIFIED,0
HOST NAME,0
NEIGHBOURHOOD GROUP,0
NEIGHBOURHOOD,0
LAT,0
LONG,0
COUNTRY,0


# *Cleaning individual columns*

In [17]:
df.head(3)

,ID,NAME,HOST ID,HOST_IDENTITY_VERIFIED,HOST NAME,NEIGHBOURHOOD GROUP,NEIGHBOURHOOD,LAT,LONG,COUNTRY,...,CANCELLATION_POLICY,ROOM TYPE,CONSTRUCTION YEAR,PRICE,SERVICE FEE,MINIMUM NIGHTS,NUMBER OF REVIEWS,REVIEW RATE NUMBER,CALCULATED HOST LISTINGS COUNT,AVAILABILITY 365
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,strict,Private room,2020.0,$966,$193,10.0,9.0,4.0,6.0,286.0
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,moderate,Entire home/apt,2007.0,$142,$28,30.0,45.0,4.0,2.0,228.0
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,moderate,Entire home/apt,2009.0,$204,$41,10.0,9.0,3.0,1.0,289.0


In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  object 
 10  COUNTRY CODE                    98509 non-null  object 
 11  INSTANT_BOOKABLE                98509 non-null  object 
 12  CANCELLATION_POLICY             9850

# SERVICE FEE

In [19]:
df['SERVICE FEE']

,SERVICE FEE
0,$193
1,$28
4,$41
5,$115
7,$212
...,...
102029,$164
102030,$91
102031,$216
102032,$21


In [20]:
df['SERVICE FEE'].unique()

array(['$193 ', '$28 ', '$41 ', '$115 ', '$212 ', '$204 ', '$58 ', '$64 ',
       '$121 ', '$116 ', '$30 ', '$219 ', '$113 ', '$74 ', '$171 ',
       '$118 ', '$168 ', '$106 ', '$13 ', '$152 ', '$105 ', '$90 ',
       '$186 ', '$66 ', '$225 ', '$19 ', '$227 ', '$32 ', '$189 ',
       '$101 ', '$23 ', '$77 ', '$190 ', '$75 ', '$231 ', '$133 ', '$35 ',
       '$82 ', '$120 ', '$21 ', '$145 ', '$65 ', '$26 ', '$89 ', '$218 ',
       '$104 ', '$44 ', '$63 ', '$79 ', '$163 ', '$22 ', '$182 ', '$165 ',
       '$37 ', '$17 ', '$48 ', '$11 ', '$131 ', '$102 ', '$240 ', '$119 ',
       '$215 ', '$191 ', '$16 ', '$233 ', '$136 ', '$170 ', '$201 ',
       '$109 ', '$123 ', '$36 ', '$12 ', '$180 ', '$238 ', '$157 ',
       '$14 ', '$198 ', '$206 ', '$122 ', '$208 ', '$140 ', '$10 ',
       '$20 ', '$110 ', '$88 ', '$98 ', '$234 ', '$151 ', '$172 ',
       '$107 ', '$138 ', '$194 ', '$139 ', '$117 ', '$31 ', '$156 ',
       '$124 ', '$175 ', '$162 ', '$187 ', '$93 ', '$51 ', '$143 ',
       '$239 '

In [21]:
df['SERVICE FEE'] = df['SERVICE FEE'].str.split('$').str[1]

In [22]:
df['SERVICE FEE'] = df['SERVICE FEE'].astype(int)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  object 
 10  COUNTRY CODE                    98509 non-null  object 
 11  INSTANT_BOOKABLE                98509 non-null  object 
 12  CANCELLATION_POLICY             9850

# PRICE

In [24]:
df['PRICE']

,PRICE
0,$966
1,$142
4,$204
5,$577
7,"$1,060"
...,...
102029,$822
102030,$455
102031,"$1,078"
102032,$103


In [25]:
df['PRICE'].unique()

array(['$966 ', '$142 ', '$204 ', ..., '$917 ', '$309 ', '$770 '],
      dtype=object)

In [26]:
df['PRICE'] = df['PRICE'].str.split('$').str[1].str.replace(',','')

In [27]:
df['PRICE'] = df['PRICE'].astype(int)

In [28]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98509 entries, 0 to 102040
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  int64  
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  int64  
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  object 
 10  COUNTRY CODE                    98509 non-null  object 
 11  INSTANT_BOOKABLE                98509 non-null  object 
 12  CANCELLATION_POLICY             9850

# ROOM TYPE

In [29]:
df['ROOM TYPE']

,ROOM TYPE
0,Private room
1,Entire home/apt
4,Entire home/apt
5,Entire home/apt
7,Private room
...,...
102029,Private room
102030,Private room
102031,Private room
102032,Private room


In [30]:
df['ROOM TYPE'].unique()

array(['Private room', 'Entire home/apt', 'Shared room', 'Hotel room'],
      dtype=object)

In [31]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()

In [32]:
encoded = encoder.fit_transform(df[['ROOM TYPE']]).toarray()

In [33]:
df_room_type = pd.DataFrame(encoded,columns=encoder.get_feature_names_out())

In [34]:
df_room_type

,ROOM TYPE_Entire home/apt,ROOM TYPE_Hotel room,ROOM TYPE_Private room,ROOM TYPE_Shared room
0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0
4,0.0,0.0,1.0,0.0
...,...,...,...,...
98504,0.0,0.0,1.0,0.0
98505,0.0,0.0,1.0,0.0
98506,0.0,0.0,1.0,0.0
98507,0.0,0.0,1.0,0.0


In [35]:
df = pd.concat([df,df_room_type],axis=1)

In [36]:
df.drop(columns=['ROOM TYPE'],inplace=True)

In [37]:
df.head(3)

,ID,NAME,HOST ID,HOST_IDENTITY_VERIFIED,HOST NAME,NEIGHBOURHOOD GROUP,NEIGHBOURHOOD,LAT,LONG,COUNTRY,...,SERVICE FEE,MINIMUM NIGHTS,NUMBER OF REVIEWS,REVIEW RATE NUMBER,CALCULATED HOST LISTINGS COUNT,AVAILABILITY 365,ROOM TYPE_Entire home/apt,ROOM TYPE_Hotel room,ROOM TYPE_Private room,ROOM TYPE_Shared room
0,1001254.0,Clean & quiet apt home by the park,8.001449e+10,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,193.0,10.0,9.0,4.0,6.0,286.0,0.0,0.0,1.0,0.0
1,1002102.0,Skylit Midtown Castle,5.233517e+10,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,28.0,30.0,45.0,4.0,2.0,228.0,1.0,0.0,0.0,0.0
4,1003689.0,Entire Apt: Spacious Studio/Loft by central park,9.203760e+10,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,41.0,10.0,9.0,3.0,1.0,289.0,0.0,0.0,1.0,0.0


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 101897 entries, 0 to 98323
Data columns (total 25 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              98509 non-null  float64
 1   NAME                            98509 non-null  object 
 2   HOST ID                         98509 non-null  float64
 3   HOST_IDENTITY_VERIFIED          98509 non-null  object 
 4   HOST NAME                       98509 non-null  object 
 5   NEIGHBOURHOOD GROUP             98509 non-null  object 
 6   NEIGHBOURHOOD                   98509 non-null  object 
 7   LAT                             98509 non-null  float64
 8   LONG                            98509 non-null  float64
 9   COUNTRY                         98509 non-null  object 
 10  COUNTRY CODE                    98509 non-null  object 
 11  INSTANT_BOOKABLE                98509 non-null  object 
 12  CANCELLATION_POLICY             9850

# CANCELLATION_POLICY

In [39]:
df['CANCELLATION_POLICY']

,CANCELLATION_POLICY
0,strict
1,moderate
4,moderate
5,flexible
7,moderate
...,...
98319,NaN
98320,NaN
98321,NaN
98322,NaN


In [40]:
df['CANCELLATION_POLICY'].unique()

array(['strict', 'moderate', 'flexible', nan], dtype=object)

In [41]:
df['CANCELLATION_POLICY'].isna().sum()

np.int64(3388)

In [42]:
df[df['CANCELLATION_POLICY'].isna()]

,ID,NAME,HOST ID,HOST_IDENTITY_VERIFIED,HOST NAME,NEIGHBOURHOOD GROUP,NEIGHBOURHOOD,LAT,LONG,COUNTRY,...,SERVICE FEE,MINIMUM NIGHTS,NUMBER OF REVIEWS,REVIEW RATE NUMBER,CALCULATED HOST LISTINGS COUNT,AVAILABILITY 365,ROOM TYPE_Entire home/apt,ROOM TYPE_Hotel room,ROOM TYPE_Private room,ROOM TYPE_Shared room
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0
12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0
15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0
98320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0
98321,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0
98322,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0


In [43]:
df.dropna(inplace=True)

In [44]:
df['CANCELLATION_POLICY'].unique()

array(['strict', 'moderate', 'flexible'], dtype=object)

In [45]:
encoded = encoder.fit_transform(df[['CANCELLATION_POLICY']]).toarray()

In [46]:
encoded

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [47]:
df_can_policy = pd.DataFrame(encoded,columns=encoder.get_feature_names_out())

In [48]:
df_can_policy

,CANCELLATION_POLICY_flexible,CANCELLATION_POLICY_moderate,CANCELLATION_POLICY_strict
0,0.0,0.0,1.0
1,0.0,1.0,0.0
2,0.0,1.0,0.0
3,1.0,0.0,0.0
4,0.0,1.0,0.0
...,...,...,...
95116,0.0,1.0,0.0
95117,0.0,0.0,1.0
95118,0.0,0.0,1.0
95119,0.0,0.0,1.0


In [49]:
df = pd.concat([df,df_can_policy],axis=True)

In [50]:
df.drop(columns=['CANCELLATION_POLICY'],inplace=True)

In [51]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98324 entries, 0 to 93581
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              95121 non-null  float64
 1   NAME                            95121 non-null  object 
 2   HOST ID                         95121 non-null  float64
 3   HOST_IDENTITY_VERIFIED          95121 non-null  object 
 4   HOST NAME                       95121 non-null  object 
 5   NEIGHBOURHOOD GROUP             95121 non-null  object 
 6   NEIGHBOURHOOD                   95121 non-null  object 
 7   LAT                             95121 non-null  float64
 8   LONG                            95121 non-null  float64
 9   COUNTRY                         95121 non-null  object 
 10  COUNTRY CODE                    95121 non-null  object 
 11  INSTANT_BOOKABLE                95121 non-null  object 
 12  CONSTRUCTION YEAR               95121

# INSTANT_BOOKABLE

In [52]:
df['INSTANT_BOOKABLE']

,INSTANT_BOOKABLE
0,False
1,False
4,False
5,True
7,False
...,...
93577,NaN
93578,NaN
93579,NaN
93580,NaN


In [53]:
df['INSTANT_BOOKABLE'].unique()

array([False, True, nan], dtype=object)

In [54]:
df.dropna(inplace=True)

In [55]:
df['INSTANT_BOOKABLE'].unique()

array([False, True], dtype=object)

In [56]:
df['INSTANT_BOOKABLE'].isna().sum()

np.int64(0)

In [57]:
df['INSTANT_BOOKABLE'] = df['INSTANT_BOOKABLE'].apply(lambda x:1 if x==True else 0)

In [58]:
df['INSTANT_BOOKABLE'].value_counts()

,count
INSTANT_BOOKABLE,
0,46208
1,45710


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91918 entries, 0 to 95120
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              91918 non-null  float64
 1   NAME                            91918 non-null  object 
 2   HOST ID                         91918 non-null  float64
 3   HOST_IDENTITY_VERIFIED          91918 non-null  object 
 4   HOST NAME                       91918 non-null  object 
 5   NEIGHBOURHOOD GROUP             91918 non-null  object 
 6   NEIGHBOURHOOD                   91918 non-null  object 
 7   LAT                             91918 non-null  float64
 8   LONG                            91918 non-null  float64
 9   COUNTRY                         91918 non-null  object 
 10  COUNTRY CODE                    91918 non-null  object 
 11  INSTANT_BOOKABLE                91918 non-null  int64  
 12  CONSTRUCTION YEAR               91918

# COUNTRY & COUNTRY CODE

In [60]:
df['COUNTRY CODE']

,COUNTRY CODE
0,US
1,US
4,US
5,US
7,US
...,...
95116,US
95117,US
95118,US
95119,US


In [61]:
df['COUNTRY CODE'].value_counts()

,count
COUNTRY CODE,
US,91918


In [62]:
df['COUNTRY'].value_counts()

,count
COUNTRY,
United States,91918


In [63]:
df['COUNTRY'] = 1
df['COUNTRY CODE'] = 1

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 91918 entries, 0 to 95120
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              91918 non-null  float64
 1   NAME                            91918 non-null  object 
 2   HOST ID                         91918 non-null  float64
 3   HOST_IDENTITY_VERIFIED          91918 non-null  object 
 4   HOST NAME                       91918 non-null  object 
 5   NEIGHBOURHOOD GROUP             91918 non-null  object 
 6   NEIGHBOURHOOD                   91918 non-null  object 
 7   LAT                             91918 non-null  float64
 8   LONG                            91918 non-null  float64
 9   COUNTRY                         91918 non-null  int64  
 10  COUNTRY CODE                    91918 non-null  int64  
 11  INSTANT_BOOKABLE                91918 non-null  int64  
 12  CONSTRUCTION YEAR               91918

# NEIGHBOURHOOD

In [65]:
df['NEIGHBOURHOOD']

,NEIGHBOURHOOD
0,Kensington
1,Midtown
4,East Harlem
5,Murray Hill
7,Bedford-Stuyvesant
...,...
95116,Middle Village
95117,Gramercy
95118,East Harlem
95119,Bedford-Stuyvesant


In [66]:
df['NEIGHBOURHOOD'].unique()

array(['Kensington', 'Midtown', 'East Harlem', 'Murray Hill',
       'Bedford-Stuyvesant', "Hell's Kitchen", 'Upper West Side',
       'Chinatown', 'South Slope', 'Bushwick', 'Flatbush', 'East Village',
       'Williamsburg', 'Harlem', 'Gowanus', 'Clinton Hill', 'Greenpoint',
       'Flatlands', 'Fort Greene', 'West Village', 'Prospect Heights',
       'DUMBO', 'Upper East Side', 'Financial District', 'Ridgewood',
       'Morningside Heights', 'Park Slope', 'Ditmars Steinway',
       'Cobble Hill', 'Flatiron District', 'Windsor Terrace',
       'Roosevelt Island', 'Lower East Side', 'Chelsea',
       'Carroll Gardens', 'Little Italy', 'Inwood', 'Kingsbridge',
       'Astoria', 'Boerum Hill', 'Washington Heights', 'Rockaway Beach',
       'Crown Heights', 'Nolita', 'Woodlawn', 'Sunnyside', 'Gramercy',
       'East New York', 'Prospect-Lefferts Gardens', 'Concourse Village',
       'Sheepshead Bay', 'Theater District', 'Emerson Hill',
       'Long Island City', 'Fort Hamilton', 'Bensonhu

In [67]:
df['NEIGHBOURHOOD'].isna().sum()

np.int64(0)

In [68]:
from sklearn.preprocessing import LabelEncoder

label = LabelEncoder()

In [69]:
encoded = label.fit_transform(df[['NEIGHBOURHOOD']])

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [70]:
encoded

array([111, 130,  62, ...,  62,  13, 194])

In [71]:
df_neighbourhood = pd.DataFrame({
    'NEIGHBOURHOOD_ENCODED' : encoded
})

In [72]:
df_neighbourhood

,NEIGHBOURHOOD_ENCODED
0,111
1,130
2,62
3,140
4,13
...,...
91913,128
91914,89
91915,62
91916,13


In [73]:
df = pd.concat([df,df_neighbourhood],axis=1)

In [74]:
df.drop(columns=['NEIGHBOURHOOD'],inplace=True)

In [75]:
df.dropna(inplace=True)

In [76]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88735 entries, 0 to 91917
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              88735 non-null  float64
 1   NAME                            88735 non-null  object 
 2   HOST ID                         88735 non-null  float64
 3   HOST_IDENTITY_VERIFIED          88735 non-null  object 
 4   HOST NAME                       88735 non-null  object 
 5   NEIGHBOURHOOD GROUP             88735 non-null  object 
 6   LAT                             88735 non-null  float64
 7   LONG                            88735 non-null  float64
 8   COUNTRY                         88735 non-null  float64
 9   COUNTRY CODE                    88735 non-null  float64
 10  INSTANT_BOOKABLE                88735 non-null  float64
 11  CONSTRUCTION YEAR               88735 non-null  float64
 12  PRICE                           88735

# NEIGHBOURHOOD GROUP

In [77]:
df['NEIGHBOURHOOD GROUP']

,NEIGHBOURHOOD GROUP
0,Brooklyn
1,Manhattan
4,Manhattan
5,Manhattan
7,Brooklyn
...,...
91913,Staten Island
91914,Manhattan
91915,Manhattan
91916,Queens


In [78]:
df['NEIGHBOURHOOD GROUP'].isna().sum()

np.int64(0)

In [79]:
df['NEIGHBOURHOOD GROUP'].unique()

array(['Brooklyn', 'Manhattan', 'brookln', 'Queens', 'Bronx',
       'Staten Island'], dtype=object)

In [80]:
encoded = label.fit_transform(df[['NEIGHBOURHOOD GROUP']])

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [81]:
encoded

array([1, 2, 2, ..., 2, 3, 1])

In [82]:
df_neighbourhood_group = pd.DataFrame({
    'NEIGHBOURHOOD GROUP_ENCODED' : encoded
})

In [83]:
df_neighbourhood_group

,NEIGHBOURHOOD GROUP_ENCODED
0,1
1,2
2,2
3,2
4,1
...,...
88730,4
88731,2
88732,2
88733,3


In [84]:
df = pd.concat([df,df_neighbourhood_group],axis=1)

In [85]:
df.drop(columns=['NEIGHBOURHOOD GROUP'],inplace=True)

In [86]:
df.dropna(inplace=True)

In [87]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 85656 entries, 0 to 88734
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              85656 non-null  float64
 1   NAME                            85656 non-null  object 
 2   HOST ID                         85656 non-null  float64
 3   HOST_IDENTITY_VERIFIED          85656 non-null  object 
 4   HOST NAME                       85656 non-null  object 
 5   LAT                             85656 non-null  float64
 6   LONG                            85656 non-null  float64
 7   COUNTRY                         85656 non-null  float64
 8   COUNTRY CODE                    85656 non-null  float64
 9   INSTANT_BOOKABLE                85656 non-null  float64
 10  CONSTRUCTION YEAR               85656 non-null  float64
 11  PRICE                           85656 non-null  float64
 12  SERVICE FEE                     85656

In [88]:
df['HOST NAME']

,HOST NAME
0,Madaline
1,Jenna
4,Lyndon
5,Michelle
7,Emma
...,...
88730,Alexa
88731,Nadia
88732,Zoë
88733,Leah


In [89]:
df['HOST NAME'].unique()

array(['Madaline', 'Jenna', 'Lyndon', ..., 'Anna B', 'Natalie And Dan',
       'Alex And Maya'], dtype=object)

In [90]:
encoded = label.fit_transform(df[['HOST NAME']])

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [91]:
encoded

array([ 7335,  5423,  7272, ..., 12946,  6779,  5625])

In [92]:
df_hostname = pd.DataFrame({
    'HOST NAME_ENCODED' : encoded
})

In [93]:
df_hostname

,HOST NAME_ENCODED
0,7335
1,5423
2,7272
3,8108
4,3408
...,...
85651,380
85652,8522
85653,12946
85654,6779


In [94]:
df = pd.concat([df,df_hostname],axis=1)

In [95]:
df.drop(columns=['HOST NAME'],inplace=True)

In [96]:
df.dropna(inplace=True)

In [97]:
df.head()

,ID,NAME,HOST ID,HOST_IDENTITY_VERIFIED,LAT,LONG,COUNTRY,COUNTRY CODE,INSTANT_BOOKABLE,CONSTRUCTION YEAR,...,ROOM TYPE_Entire home/apt,ROOM TYPE_Hotel room,ROOM TYPE_Private room,ROOM TYPE_Shared room,CANCELLATION_POLICY_flexible,CANCELLATION_POLICY_moderate,CANCELLATION_POLICY_strict,NEIGHBOURHOOD_ENCODED,NEIGHBOURHOOD GROUP_ENCODED,HOST NAME_ENCODED
0,1001254.0,Clean & quiet apt home by the park,8.001449e+10,unconfirmed,40.64749,-73.97237,1.0,1.0,0.0,2020.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,111.0,1.0,7335.0
1,1002102.0,Skylit Midtown Castle,5.233517e+10,verified,40.75362,-73.98377,1.0,1.0,0.0,2007.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,130.0,2.0,5423.0
4,1003689.0,Entire Apt: Spacious Studio/Loft by central park,9.203760e+10,verified,40.79851,-73.94399,1.0,1.0,0.0,2009.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,13.0,1.0,3408.0
5,1004098.0,Large Cozy 1 BR Apartment In Midtown East,4.549855e+10,verified,40.74767,-73.97500,1.0,1.0,1.0,2013.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,98.0,2.0,3593.0
7,1005202.0,BlissArtsSpace!,9.082184e+10,unconfirmed,40.68688,-73.95596,1.0,1.0,0.0,2009.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,36.0,2.0,8234.0


In [98]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82612 entries, 0 to 85655
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              82612 non-null  float64
 1   NAME                            82612 non-null  object 
 2   HOST ID                         82612 non-null  float64
 3   HOST_IDENTITY_VERIFIED          82612 non-null  object 
 4   LAT                             82612 non-null  float64
 5   LONG                            82612 non-null  float64
 6   COUNTRY                         82612 non-null  float64
 7   COUNTRY CODE                    82612 non-null  float64
 8   INSTANT_BOOKABLE                82612 non-null  float64
 9   CONSTRUCTION YEAR               82612 non-null  float64
 10  PRICE                           82612 non-null  float64
 11  SERVICE FEE                     82612 non-null  float64
 12  MINIMUM NIGHTS                  82612

# HOST_IDENTITY_VERIFIED

In [99]:
df['HOST_IDENTITY_VERIFIED']

,HOST_IDENTITY_VERIFIED
0,unconfirmed
1,verified
4,verified
5,verified
7,unconfirmed
...,...
85651,unconfirmed
85652,verified
85653,verified
85654,unconfirmed


In [100]:
df['HOST_IDENTITY_VERIFIED'].unique()

array(['unconfirmed', 'verified'], dtype=object)

In [101]:
df['HOST_IDENTITY_VERIFIED'].isna().sum()

np.int64(0)

In [102]:
df['HOST_IDENTITY_VERIFIED'] = df['HOST_IDENTITY_VERIFIED'].apply(lambda x:1 if x=='verified' else 0)

In [103]:
df['HOST_IDENTITY_VERIFIED'].value_counts()

,count
HOST_IDENTITY_VERIFIED,
1,41341
0,41271


In [104]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 82612 entries, 0 to 85655
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              82612 non-null  float64
 1   NAME                            82612 non-null  object 
 2   HOST ID                         82612 non-null  float64
 3   HOST_IDENTITY_VERIFIED          82612 non-null  int64  
 4   LAT                             82612 non-null  float64
 5   LONG                            82612 non-null  float64
 6   COUNTRY                         82612 non-null  float64
 7   COUNTRY CODE                    82612 non-null  float64
 8   INSTANT_BOOKABLE                82612 non-null  float64
 9   CONSTRUCTION YEAR               82612 non-null  float64
 10  PRICE                           82612 non-null  float64
 11  SERVICE FEE                     82612 non-null  float64
 12  MINIMUM NIGHTS                  82612

# NAME

In [105]:
df['NAME']

,NAME
0,Clean & quiet apt home by the park
1,Skylit Midtown Castle
4,Entire Apt: Spacious Studio/Loft by central park
5,Large Cozy 1 BR Apartment In Midtown East
7,BlissArtsSpace!
...,...
85651,Huge New 1700sf Brownstone Duplex
85652,BrownBrick in Park Slope Brownstone
85653,1 Bedroom June Through July
85654,Sun Drenched Studio


In [106]:
df['NAME'].unique()

array(['Clean & quiet apt home by the park', 'Skylit Midtown Castle',
       'Entire Apt: Spacious Studio/Loft by central park', ...,
       'Top location amazing cozy 1BR',
       'Comfy Room in Large E Williamsburg Apartment',
       '⚜ Two-Floor Vibrant 2BR Sanctuary with Patio! ⚜'], dtype=object)

In [107]:
df['NAME'].value_counts()

,count
NAME,
Water View King Bed Hotel Room,25
Home away from home,25
#NAME?,20
New york Multi-unit building,18
Beautiful 2 Double Bed Hotel Room,18
...,...
3-bdrm Newly Renovated Townhouse.,1
"Designer Studio / 1BR in Williamsburg, Nice Views",1
Barclays Place - An Idyllic One Bedroom Apartment,1


In [108]:
encoded = label.fit_transform(df[['NAME']])

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [109]:
encoded

array([15362, 46571, 23098, ...,   899, 51368, 13693])

In [110]:
df_name = pd.DataFrame({
    'NAME_ENCODED' : encoded
})

In [111]:
df_name

,NAME_ENCODED
0,15362
1,46571
2,23098
3,30086
4,9979
...,...
82607,28108
82608,12357
82609,899
82610,51368


In [112]:
df = pd.concat([df,df_name],axis=1)

In [113]:
df.drop(columns=['NAME'],inplace=True)

In [114]:
df.dropna(inplace=True)

In [115]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 79731 entries, 0 to 82611
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              79731 non-null  float64
 1   HOST ID                         79731 non-null  float64
 2   HOST_IDENTITY_VERIFIED          79731 non-null  float64
 3   LAT                             79731 non-null  float64
 4   LONG                            79731 non-null  float64
 5   COUNTRY                         79731 non-null  float64
 6   COUNTRY CODE                    79731 non-null  float64
 7   INSTANT_BOOKABLE                79731 non-null  float64
 8   CONSTRUCTION YEAR               79731 non-null  float64
 9   PRICE                           79731 non-null  float64
 10  SERVICE FEE                     79731 non-null  float64
 11  MINIMUM NIGHTS                  79731 non-null  float64
 12  NUMBER OF REVIEWS               79731

In [116]:
df.duplicated().sum()

np.int64(0)

In [117]:
df.head()

,ID,HOST ID,HOST_IDENTITY_VERIFIED,LAT,LONG,COUNTRY,COUNTRY CODE,INSTANT_BOOKABLE,CONSTRUCTION YEAR,PRICE,...,ROOM TYPE_Hotel room,ROOM TYPE_Private room,ROOM TYPE_Shared room,CANCELLATION_POLICY_flexible,CANCELLATION_POLICY_moderate,CANCELLATION_POLICY_strict,NEIGHBOURHOOD_ENCODED,NEIGHBOURHOOD GROUP_ENCODED,HOST NAME_ENCODED,NAME_ENCODED
0,1001254.0,8.001449e+10,0.0,40.64749,-73.97237,1.0,1.0,0.0,2020.0,966.0,...,0.0,1.0,0.0,0.0,0.0,1.0,111.0,1.0,7335.0,15362.0
1,1002102.0,5.233517e+10,1.0,40.75362,-73.98377,1.0,1.0,0.0,2007.0,142.0,...,0.0,0.0,0.0,0.0,1.0,0.0,130.0,2.0,5423.0,46571.0
4,1003689.0,9.203760e+10,1.0,40.79851,-73.94399,1.0,1.0,0.0,2009.0,204.0,...,0.0,1.0,0.0,0.0,1.0,0.0,13.0,1.0,3408.0,9979.0
5,1004098.0,4.549855e+10,1.0,40.74767,-73.97500,1.0,1.0,1.0,2013.0,577.0,...,0.0,1.0,0.0,0.0,0.0,1.0,98.0,2.0,3593.0,30123.0
7,1005202.0,9.082184e+10,0.0,40.68688,-73.95596,1.0,1.0,0.0,2009.0,1060.0,...,0.0,0.0,0.0,1.0,0.0,0.0,36.0,2.0,8234.0,21045.0


In [119]:
# prompt: create a new csv file with updated datatha has new data

# Assuming df is the DataFrame from the provided code
df.to_csv('updated_airbnb_data.csv', index=False)
